# Initial visualization of GRACE mascons
## Dataset
GSFC mascons: [data](https://earth.gsfc.nasa.gov/geo/data/grace-mascons), [documentation](https://earth.gsfc.nasa.gov/sites/default/files/2022-05/gsfc_mascons_hdf5_format_rl06v2.pdf)

## Environments and packages


In [1]:
grace_filename = 'gsfc.glb_.200204_202211_rl06v2.0_obp-ice6gd.h5'
grace_filepath = '/global/scratch/users/ann_scheliga/'

In [2]:
import numpy as np
import pandas as pd
import h5py

In [4]:
def h5group_to_df(h5group):
    """
    Input
    -----
    h5group: h5py._hl.group.Group
    
    Output
    ------
    """

In [5]:
# (works) mascon metadata
mascon_cols = list(f['mascon'])       # grab dataset names in mascon group
mascon_cols.remove('location_legend') # remove unused dataset name
mascon_df = pd.DataFrame()            # create empty pd.DataFrame
for key in mascon_cols:               # fill df
    mascon_df[key] = np.array(f['mascon'][key]).T.squeeze()

In [29]:
# (works) WE solutions
cmwe = pd.DataFrame(f['solution']['cmwe'])
cmwe.loc[land_bool,:]

,0,1,2,3,4,5,6,7,8,9,...,205,206,207,208,209,210,211,212,213,214
1425,3.029239,2.304514,-16.058898,-15.821627,-10.138998,-2.887646,1.875435,4.068886,6.680802,10.872631,...,4.255870,3.765805,6.096908,4.851900,-1.047893,-3.948667,-6.552598,-8.281110,-12.061341,-7.967980
1426,2.227589,-2.122726,-14.004051,-12.736436,-9.077046,-3.271939,1.995051,4.621046,6.651718,10.931204,...,1.786547,1.308632,3.867905,2.781919,-3.339602,-5.824893,-8.388326,-9.125167,-12.254486,-8.569515
1427,0.132170,-6.730445,-12.492522,-9.915949,-7.367986,-3.994733,1.759553,4.212027,6.121970,9.922467,...,1.061422,0.352646,2.960094,2.176743,-4.037788,-5.967311,-8.026859,-8.056577,-10.442594,-7.505179
1428,1.588050,3.693477,-17.490956,-16.721249,-9.635691,-3.628682,1.175182,2.792983,5.994315,10.543806,...,9.855744,8.996314,11.532702,9.401525,3.942752,0.704760,-1.450398,-4.479732,-9.582990,-4.886631
1429,1.568100,-0.369924,-16.324466,-14.910996,-9.756903,-3.158430,1.663575,3.598481,6.256994,10.918030,...,6.932148,5.750392,8.231167,6.522527,0.674849,-1.937151,-4.276239,-6.296543,-10.255156,-6.219922
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13023,-2.376923,-2.542053,8.122327,3.775422,6.518201,7.755171,5.163998,-5.997924,-3.079955,-4.903888,...,-9.036207,-15.338093,-7.615341,-3.463772,-4.218099,-3.826824,-5.452903,2.972445,8.417412,2.009305
13024,-2.781673,-2.937394,7.684555,5.734322,6.786544,8.251943,4.043084,-5.908650,-3.117862,-5.441375,...,-8.880043,-17.248843,-8.450040,-3.334791,-4.365773,-4.696816,-5.810918,3.407042,8.656171,1.570265
13025,-3.263036,-3.911417,6.993020,1.199251,6.689715,8.272824,6.366393,-6.359607,-3.444465,-4.969424,...,-6.925252,-13.154171,-5.551532,-2.858411,-3.807784,-0.787634,-4.969065,3.288228,8.844646,2.960390
13026,-3.398121,-4.659551,7.238876,2.888916,7.568851,9.302146,5.955749,-6.469833,-3.461794,-5.496231,...,-8.753762,-16.102850,-7.897494,-4.035575,-4.591893,-3.696783,-5.939988,2.868247,8.191702,1.723358


In [7]:
# (works) dates
# REFERENCE DATE (aka 2002 Jan 0)
start_date = pd.Timestamp('2001-12-31')

time_cols = list(f['time'])                # grab dataset names in time group
time_cols.remove('list_ref_days_solution') # remove unused dataset name
time_df = pd.DataFrame()                   # create empty pd.DataFrame for reference dates
for key in time_cols[2:-1]:                # fill df with days since reference day
    time_df[key] = np.array(f['time'][key]).T.squeeze()

date_df = time_df.apply(lambda x: pd.to_datetime(x, unit='D',origin=start_date),axis=1)
date_df.columns = ['date_first','date_last','date_middle']
date_df[['year_middle','doy_middle','frac_year_middle']] = pd.DataFrame(f['time']['yyyy_doy_yrplot_middle']).T

In [19]:
# (works) uncertainty
uncertainty_cols = list(f['uncertainty'])
uncertainty_df = pd.DataFrame()
for key in uncertainty_cols[:-1]:
    uncertainty_df[key] = np.array(f['uncertainty'][key]).T.squeeze()
noise_df = pd.DataFrame(f['uncertainty']['noise_2sigma'])
uncertainty_df = pd.concat([uncertainty_df, noise_df],axis=1)

count    41168.000000
mean         1.004000
std          1.050979
min          0.206016
25%          0.343326
50%          0.540978
75%          1.425697
max         26.885994
Name: leakage_2sigma, dtype: float64
count    41168.000000
mean         0.000109
std          0.768421
min        -28.177664
25%         -0.051017
50%         -0.000906
75%          0.049189
max         28.599211
Name: leakage_trend, dtype: float64


In [30]:
land_bool = mascon_df['location'] == 80
uncertainty_df.loc[land_bool,:]

,leakage_2sigma,leakage_trend,0,1,2,3,4,5,6,7,...,205,206,207,208,209,210,211,212,213,214
1425,1.459921,0.078367,2.043895,2.047837,1.603160,2.123730,2.017953,1.859288,2.219555,2.527967,...,2.579555,2.838867,2.823086,2.441536,2.520600,2.286924,3.390276,3.096659,2.165510,1.588099
1426,1.584509,0.131458,2.174324,2.178518,1.705464,2.259254,2.146727,1.977936,2.361194,2.689287,...,2.744167,3.020027,3.003239,2.597340,2.681450,2.432862,3.606624,3.294270,2.303700,1.689443
1427,1.912270,0.084733,2.535013,2.539903,1.988376,2.634032,2.502837,2.306048,2.752882,3.135401,...,3.199385,3.521005,3.501432,3.028201,3.126264,2.836439,4.204911,3.840742,2.685850,1.969697
1428,1.569446,-0.150867,2.117003,2.121086,1.660503,2.199694,2.090133,1.925792,2.298946,2.618390,...,2.671823,2.940410,2.924065,2.528867,2.610759,2.368725,3.511543,3.207424,2.242968,1.644904
1429,1.596565,-0.046444,2.159998,2.164164,1.694227,2.244368,2.132582,1.964904,2.345637,2.671568,...,2.726086,3.000128,2.983451,2.580226,2.663782,2.416832,3.582860,3.272564,2.288521,1.678311
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13023,3.590233,0.017526,5.204361,5.214399,4.082119,5.407645,5.138304,4.734296,5.651644,6.436951,...,6.568310,7.228593,7.188411,6.216871,6.418192,5.823184,8.632645,7.885010,5.514027,4.043770
13024,3.792165,-0.071378,5.256329,5.266467,4.122881,5.461643,5.189612,4.781570,5.708078,6.501227,...,6.633897,7.300774,7.260190,6.278949,6.482281,5.881331,8.718846,7.963745,5.569088,4.084149
13025,4.956585,-0.181954,7.474509,7.488927,5.862745,7.766467,7.379638,6.799402,8.116898,9.244758,...,9.433415,10.381715,10.324005,8.928678,9.217816,8.363265,12.398216,11.324462,7.919253,5.807668
13026,4.120352,-0.164269,6.447709,6.460146,5.057359,6.699560,6.365871,5.865344,7.001851,7.974772,...,8.137513,8.955542,8.905759,7.702114,7.951532,7.214373,10.695029,9.768780,6.831357,5.009848


In [37]:
def land_subset(input):
    
cmwe

date_middle,2002-04-18,2002-05-11,2002-08-17,2002-09-16,2002-10-17,2002-11-16,2002-12-17,2003-01-17,2003-02-15,2003-03-17,...,2022-02-15,2022-03-17,2022-04-16,2022-05-17,2022-06-16,2022-07-17,2022-08-17,2022-09-16,2022-10-17,2022-11-16
labels,,,,,,,,,,,,,,,,,,,,,
1,54.691892,78.133855,44.547898,43.710999,47.730855,45.079064,44.879136,46.301053,40.554585,47.421868,...,-169.934978,-172.100230,-190.539332,-190.096065,-188.894206,-190.234859,-208.976192,-177.947764,-184.212459,-181.977580
2,52.269963,71.443174,37.124612,36.021451,39.218568,38.015584,37.367627,39.105039,34.496736,37.858018,...,-163.581218,-166.974879,-180.707209,-180.007762,-182.461299,-181.030940,-195.847606,-178.194224,-178.577739,-180.615199
3,43.002050,55.813134,26.720418,25.472089,27.558011,27.436334,26.885598,29.419812,24.898534,28.375791,...,-139.183722,-146.594695,-160.948928,-153.056136,-160.392897,-155.370162,-172.637053,-155.480590,-152.158919,-157.602446
4,36.156027,47.043185,36.438498,39.869586,43.686755,42.355116,40.124064,40.108393,37.137099,41.672099,...,-134.247106,-131.784139,-139.022039,-148.329549,-141.484365,-146.231389,-146.716390,-142.244542,-147.246419,-141.862142
5,43.983200,58.883858,38.699711,39.441243,43.129734,41.657856,40.265541,41.269135,38.071015,42.671891,...,-143.499961,-142.712883,-151.648597,-162.157463,-155.846046,-160.167884,-163.340126,-157.344795,-163.639900,-159.185417
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41164,-5.398593,-6.702841,0.730257,1.939265,-1.375402,-1.451277,3.618848,10.318260,-1.976756,2.097339,...,-1.090784,1.452377,6.764794,14.214471,2.259454,11.267843,8.276225,16.192170,13.999838,23.389499
41165,-5.143282,-3.585495,1.357815,1.995537,-2.264055,-2.489200,2.521876,10.110209,-1.066093,-0.819059,...,3.754460,8.292587,11.895864,17.609335,4.120021,12.057978,11.484878,17.089279,14.599531,24.619624
41166,-3.280975,0.173697,2.577265,4.236445,-1.615919,-2.939253,1.151613,9.196100,0.658966,-4.111098,...,7.080587,13.582077,14.187294,16.996234,6.388536,9.676683,13.464657,16.698427,14.130370,22.931265
